<a href="https://colab.research.google.com/github/inbox00/new/blob/main/mutli-tor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3-libtorrent

import libtorrent as lt
from google.colab import drive
from google.colab import files
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create libtorrent session
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# Upload multiple torrent files
uploaded = files.upload()

# Parse and add torrents to session
for filename, torrent_content in uploaded.items():
    info = lt.torrent_info(torrent_content)
    print(f"Files in the torrent '{filename}':")
    for i, f in enumerate(info.files()):
        print(i, f.path)

    # Choose the index of the specific file you want to download or choose 'all'
    file_index = input(f"Enter the index of the file you want to download from '{filename}' or 'all' for all files: ")

    # Add the selected file(s) to the libtorrent session for downloading
    params = {"save_path": "/content/drive/My Drive/Torrent"}
    params["ti"] = info
    params["file_priorities"] = [1] * info.num_files() if file_index == 'all' else [0] * info.num_files()
    if file_index != 'all':
        file_index = int(file_index)
        params["file_priorities"][file_index] = 1

    downloads.append(ses.add_torrent(params))

# Monitor download progress
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")

            # Move downloaded file(s) to Google Drive
            if file_index == 'all':
                for file in info.files():
                    downloaded_file_path = os.path.join(params["save_path"], file.path)
                    target_drive_path = '/content/drive/My Drive/'
                    target_path = os.path.join(target_drive_path, os.path.basename(downloaded_file_path))
                    os.rename(downloaded_file_path, target_path)
                    print("File moved to Google Drive:", target_path)
            else:
                downloaded_file_path = os.path.join(params["save_path"], info.files()[file_index].path)
                target_drive_path = '/content/drive/My Drive/'
                target_path = os.path.join(target_drive_path, os.path.basename(downloaded_file_path))
                os.rename(downloaded_file_path, target_path)
                print("File moved to Google Drive:", target_path)

    time.sleep(1)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-14-e55c2ad0deb0>:13: DeprecationWarning: listen_on() is deprecated
  ses.listen_on(6881, 6891)


<ipython-input-14-e55c2ad0deb0>:23: DeprecationWarning: __iter__ is deprecated
  for i, f in enumerate(info.files()):


Saving renault-dialogys-v4.72-eng_autosoftos.com.torrent to renault-dialogys-v4.72-eng_autosoftos.com.torrent
Saving renaul-dialogsys-3_53-working-ok-old-dacia-models_autosoftos_com.torrent to renaul-dialogsys-3_53-working-ok-old-dacia-models_autosoftos_com (3).torrent
Saving renault-dialogys-4.78-multi-4.78-multi-rus_autosoftos.com.torrent to renault-dialogys-4.78-multi-4.78-multi-rus_autosoftos.com (2).torrent
Saving renault-dialogys-4_9-eng-rus_autosoftos_com.torrent to renault-dialogys-4_9-eng-rus_autosoftos_com (1).torrent
Files in the torrent 'renault-dialogys-v4.72-eng_autosoftos.com.torrent':
0 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD3.iso
1 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD2.iso
2 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD5.iso
3 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD4.iso
4 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD1.iso
5 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD6.iso
6 Renault Dialogys v4_72 EN/Dialogys_v4_72_En_DVD0.iso
7 Renau

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

<ipython-input-14-e55c2ad0deb0>:69: DeprecationWarning: is_seed() is deprecated
  if not download.is_seed():
<ipython-input-14-e55c2ad0deb0>:74: DeprecationWarning: name() is deprecated
  download.name(),
